In [10]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [26]:
# set up the URL for scraping
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 101.0.4951
[WDM] - Get LATEST chromedriver version for 101.0.4951 google-chrome
[WDM] - Driver [/Users/veronicalobkina/.wdm/drivers/chromedriver/mac64/101.0.4951.41/chromedriver] found in cache


In [12]:
# Visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [13]:
# set up the html parser
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')

In [14]:
# begin scraping
slide_elem.find('div', class_='content_title')

<div class="content_title">The Launch Is Approaching for NASA's Next Mars Rover, Perseverance</div>

In [15]:
# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

"The Launch Is Approaching for NASA's Next Mars Rover, Perseverance"

In [16]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

"The Red Planet's surface has been visited by eight NASA spacecraft. The ninth will be the first that includes a roundtrip ticket in its flight plan. "

### Featured Images

In [17]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [18]:
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [19]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [20]:
# Find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars2.jpg'

In [21]:
# Use the base URL to create an absolute URL
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars2.jpg'

In [22]:
df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.columns=['description', 'Mars', 'Earth']
df.set_index('description', inplace=True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [23]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

In [24]:
browser.quit()

## D1: Scrape High-Resolution Mars’ Hemisphere Images and Titles

### Hemispheres

In [27]:
# 1. Use browser to visit the URL 
url = 'https://marshemispheres.com/'

browser.visit(url)


html = browser.html
hem_soup = soup(html, 'html.parser')
# collect all class = items
items = hem_soup.find_all('div', class_='item')

In [28]:
# 2. Create a list to hold the images and titles.
hemisphere_image_urls = []

# 3. Write code to retrieve the image urls and titles for each hemisphere.
for z in items:
    #empty dictionary to hold scrape
    hemisphere = {}
    # find the titles
    titles = z.find('h3').text
    link_ref = z.find('a', class_='itemLink product-item')['href']
    
    # concat to get the full link
    browser.visit(url + link_ref)
    
    image_html = browser.html
    image_soup = soup(image_html, 'html.parser')
    download = image_soup.find('div', class_='downloads')
    image_url = download.find('a')['href']
    
    print(titles)
    print(img_url)
    
    # add titles and image url to list using append
    hemisphere['img_url'] = img_url
    hemisphere['title'] = titles
    hemisphere_image_urls.append(hemisphere)

Cerberus Hemisphere Enhanced
https://spaceimages-mars.com/image/featured/mars2.jpg
Schiaparelli Hemisphere Enhanced
https://spaceimages-mars.com/image/featured/mars2.jpg
Syrtis Major Hemisphere Enhanced
https://spaceimages-mars.com/image/featured/mars2.jpg
Valles Marineris Hemisphere Enhanced
https://spaceimages-mars.com/image/featured/mars2.jpg


In [29]:
# 4. Print the list that holds the dictionary of each image url and title.
hemisphere_image_urls

[{'img_url': 'https://spaceimages-mars.com/image/featured/mars2.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://spaceimages-mars.com/image/featured/mars2.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://spaceimages-mars.com/image/featured/mars2.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://spaceimages-mars.com/image/featured/mars2.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [30]:
# 5. Quit the browser
browser.quit()